In [1]:
from astropy.io import fits, ascii
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18
import numpy as np
import tqdm
from astropy.coordinates import SpectralCoord
from astropy.table import Table

In [2]:
catalog = ascii.read('/home/manth145/data/Merger_Ionization_State/SDSS_DR17_Matched_Information.csv', format='csv')
cleaned_catalog = catalog[np.where((catalog['clean']==1) & (catalog['type']==3))]
redshift_cut = cleaned_catalog[np.where((cleaned_catalog['z']>=0.01) & (cleaned_catalog['z']<=0.055))]
mass_cut = redshift_cut[np.where(redshift_cut['logMass']>=8.5)]

primary_catalog = mass_cut[np.where((mass_cut['oiii_5007_flux']/mass_cut['oiii_5007_flux_err']>=3) 
                                    & ((mass_cut['oii_3726_flux']/mass_cut['oii_3726_flux_err']>=3)))]
secondary_catalog = redshift_cut[np.where(redshift_cut['logMass']>=8)]

secondary_coords_array = SkyCoord(secondary_catalog['ra'], secondary_catalog['dec'], unit=u.deg)

/tmp/ipykernel_404224/3482134323.py:6: RuntimeWarning: invalid value encountered in true_divide
  primary_catalog = mass_cut[np.where((mass_cut['oiii_5007_flux']/mass_cut['oiii_5007_flux_err']>=3)
/tmp/ipykernel_404224/3482134323.py:7: RuntimeWarning: invalid value encountered in true_divide
  & ((mass_cut['oii_3726_flux']/mass_cut['oii_3726_flux_err']>=3)))]


In [3]:
pair_catalog = Table(names=['primary_ObjID', 'secondary_ObjID', 'primary_z', 'secondary_z', 'proj_sep', 'rel_velocity', 'mass_ratio'],
                    dtype=[int, int, float, float, float, float, float])

In [4]:
for each_primary in tqdm.tqdm(primary_catalog):
    coords_primary = SkyCoord(each_primary['ra']*u.deg, each_primary['dec']*u.deg, frame='icrs')
    coord_array_primary = SkyCoord([each_primary['ra']*u.deg], [each_primary['dec']*u.deg], frame='icrs')
    spec_coord_primary = SpectralCoord(4000, u.Angstrom, redshift=each_primary['z'])
    _, specific_search_objects,_,_ = secondary_coords_array.search_around_sky(coord_array_primary, seplimit=12*u.arcmin)
    for each_secondary in secondary_catalog[specific_search_objects]:
        if (each_primary['ObjID'] != each_secondary['ObjID']) and (each_primary['logMass']>=each_secondary['logMass']):
            coords_secondary = SkyCoord(each_secondary['ra']*u.deg, each_secondary['dec']*u.deg, frame='icrs')
            spec_coord_secondary = SpectralCoord(4000, u.Angstrom, redshift=each_secondary['z'])
            sep = coords_primary.separation(coords_secondary).arcsecond
            scale = Planck18.kpc_comoving_per_arcmin(each_primary['z']).value/60.0
            proj_sep = sep * scale
            rel_velocity = np.abs(spec_coord_primary.radial_velocity.value - spec_coord_secondary.radial_velocity.value)
            mass_ratio = each_primary['logMass'] - each_secondary['logMass']
            if proj_sep <=150 and rel_velocity<=1000:
                pair_catalog.add_row([each_primary['ObjID'], each_secondary['ObjID'], 
                                      each_primary['z'], each_secondary['z'], proj_sep, rel_velocity, mass_ratio])
            
#     break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52700/52700 [06:03<00:00, 145.03it/s]


In [5]:
ascii.write(pair_catalog, 'pair_catalog.csv', format='csv', overwrite=True)

In [8]:
secondary_coords = SkyCoord(secondary_catalog['ra'], secondary_catalog['dec'], unit=u.deg)

In [16]:
secondary_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(229.95682791, 7.81281151), (230.02044927, 7.82496405),
     (230.07385071, 7.7345885 ), ..., (229.67020891, 7.82983421),
     (229.71875656, 7.80551449), (229.77133456, 7.816241  )]>

In [17]:
coords_primary

<SkyCoord (ICRS): (ra, dec) in deg
    (229.95682791, 7.81281151)>

<SkyCoord (ICRS): (ra, dec) in deg
    [(229.95682791, 7.81281151)]>

In [19]:
something = secondary_coords.search_around_sky(SkyCoord([each_primary['ra']*u.deg], [each_primary['dec']*u.deg], frame='icrs'), seplimit=12*u.arcmin)

In [29]:
secondary_catalog[something[1]]

modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,ra,dec,z,ObjID,mode,type,clean,specObjID,Flux_OIII_4958,Flux_OIII_4958_Err,FLux_OIII_5006,FLux_OIII_5006_Err,Flux_OII_3726,Flux_OII_3726_Err,Flux_OII_3728,Flux_OII_3728_Err,oiii_4959_flux,oiii_4959_flux_err,oiii_5007_flux,oiii_5007_flux_err,oii_3729_flux,oii_3729_flux_err,oii_3726_flux,oii_3726_flux_err,logMass,metallicity,age,ssfr,Column1,minlogMass,maxLogMass,SFR,minSFR,maxSFR,Column2
float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
18.42497,16.46613,15.56528,15.14275,14.7951,229.956827905323,7.81281151181592,0.0425399,1237662197747745143,1,3,1,2046998256977733632,15.1694,5.202966,43.34115,14.86562,162.7156,8668.07,83.20232,12623.88,2.195039,3.944313,20.52398,4.17063,8.500624,6.295256,23.5517,6.658473,10.6245435100394,0.00769999995827675,3.41003704071045,-3.35963487625122,10.49,10.44,10.82,0.0,0.0,0.1,0.04
18.42605,16.77179,15.95624,15.55291,15.22884,230.020449266655,7.8249640540335,0.04858921,1237662197747810509,1,3,1,2047008977216104448,2.54,3.480551,7.257142,9.944433,32.70702,14.09445,28.81954,14.82691,4.886132,3.886356,8.983654,3.882987,14.52591,6.793278,16.43024,7.012192,10.527184246644,0.00769999995827675,3.50055408477783,-1.64312648773193,10.3,9.87,10.3,0.5,0.0,0.5,0.04
18.32344,16.41611,15.56927,15.17142,14.83763,230.073850709502,7.73458849981151,0.04826854,1237662197747810581,1,3,1,2047010626483546112,19.41872,22.97698,55.48206,65.64853,18.42138,24.8859,63.67159,96.90556,-2.144605,4.111876,-1.197808,4.011215,4.013726,7.089595,-7.487408,6.840191,10.6211549418894,0.00769999995827675,2.30776882171631,-4.74274015426636,10.48,10.4,10.82,0.0,0.0,0.0,0.04
19.23968,17.50391,16.67721,16.28032,15.96635,230.12442382008,7.74330536885284,0.04342559,1237662197747876094,1,3,1,2047008702338197504,2.317399,3.073234,6.621141,8.780669,6.759706,0.0,20.28467,15.74386,0.495332,2.124155,3.201239,2.135891,2.677525,3.586457,-7.045466,3.395661,10.0623540645977,0.00769999995827675,2.24202561378479,-3.40160775184631,10.05,9.91,10.3,0.0,0.0,0.1,0.02
19.383,17.76669,17.02072,16.67171,16.38346,229.979520387313,7.8848999384746,0.04368504,1237662238559633634,1,3,1,2047007602826569728,0.8643174,3.35695,2.469478,9.591287,5.522397,18.33613,9.157022,19.73397,-1.601528,1.669952,2.927187,1.713424,3.155766,3.445201,-1.580159,3.202087,9.73223140323893,0.00960000045597553,2.16739392280579,-2.55261540412903,9.21,9.2,9.46,0.0,0.0,0.0,0.01
19.90191,18.09789,17.26624,16.81353,16.49999,230.117006583752,7.91962126094432,0.04737807,1237662238559699173,1,3,1,2047010901361453056,3.151606,3.346088,9.00459,9.560251,71.70338,287.5388,2.903637,7723.571,0.5847952,1.785761,1.47943,1.891614,1.058998,4.14849,12.51197,4.241342,10.2847171088276,0.00769999995827675,8.18727207183838,-1.71574354171753,9.49,9.42,10.2,0.0,0.0,0.1,0.04
17.55678,15.6935,14.80786,14.37943,14.04768,229.952270715335,7.64351458129482,0.04504822,1237662264323276903,1,3,1,2047022995989358592,0.0,0.0,0.0,0.0,678.1226,102.2245,0.0,0.0,-3.380143,6.269952,-2.422726,6.110695,-17.61811,17.30406,20.67467,17.34901,10.9303692902658,0.00769999995827675,2.90692257881165,-4.53546094894409,10.67,10.56,11.07,0.0,0.0,0.4,0.04
19.57024,17.78923,16.93512,16.53202,16.2037,229.872501290654,7.75109488591342,0.04516993,1237662264323276989,1,3,1,2047000181123082240,10.66989,7.84012,30.4854,22.40034,241.8281,72.10158,3.767811,0.0,-12.11991,6.170697,11.65344,6.178532,-13.0267,29.32568,11.43088,29.37355,10.0179470964777,0.00769999995827675,2.30776882171631,-4.74274015426636,10.04,9.92,10.26,0.0,0.0,0.1,0.02
19.17047,18.14644,17.69647,17.42855,17.20314,229.936938003302,7.62825612515211,0.0446186,1237662264323277094,1,3,1,2047020522088196096,11.33737,2.230657,32.39249,6.373305,138.5244,48.52168,27.40748,52.67923,4.923667,

In [5]:
sep * scale

209.0578808875126

In [6]:
rel_velocity

1731.0672104809892

In [30]:
150/(Planck18.kpc_comoving_per_arcmin(0.055).value/60.0)

128.62738506186437

In [13]:
150/Planck18.kpc_comoving_per_arcmin(0.01).value

11.665227175746203

In [7]:
each_primary

modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,ra,dec,z,ObjID,mode,type,clean,specObjID,Flux_OIII_4958,Flux_OIII_4958_Err,FLux_OIII_5006,FLux_OIII_5006_Err,Flux_OII_3726,Flux_OII_3726_Err,Flux_OII_3728,Flux_OII_3728_Err,oiii_4959_flux,oiii_4959_flux_err,oiii_5007_flux,oiii_5007_flux_err,oii_3729_flux,oii_3729_flux_err,oii_3726_flux,oii_3726_flux_err,logMass,metallicity,age,ssfr,Column1,minlogMass,maxLogMass,SFR,minSFR,maxSFR,Column2
float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
18.42497,16.46613,15.56528,15.14275,14.7951,229.956827905323,7.81281151181592,0.0425399,1237662197747745143,1,3,1,2046998256977733632,15.1694,5.202966,43.34115,14.86562,162.7156,8668.07,83.20232,12623.88,2.195039,3.944313,20.52398,4.17063,8.500624,6.295256,23.5517,6.658473,10.6245435100394,0.00769999995827675,3.41003704071045,-3.35963487625122,10.49,10.44,10.82,0.0,0.0,0.1,0.04


In [8]:
dummy = SpectralCoord(4000, u.Angstrom, redshift=each_primary['z'])
dummy2 = SpectralCoord(4000, u.Angstrom, redshift=each_secondary['z'])

In [14]:
np.abs(dummy.radial_velocity.value - dummy2.radial_velocity.value)

1731.0672104809892

In [19]:
np.abs(each_primary['z'] - each_secondary['z']) * speed_of_light/1000

<Quantity 1813.5375141 m / s>

In [17]:
from astropy.constants import c as speed_of_light